In [1]:
from tqdm import tqdm
import numpy as np
import networkx as nx
import tsplib95
import matplotlib.pyplot as plt
import pandas as pd
import math

In [8]:
sym_data={
    'filename':[
        'a280',
        'att48',
        'berlin52',
        'bier127',
        'burma14',
        'ch130',
        'ch150',
        'eil51',
        'eil101',
        'eil76',
        'kroC100',
        'kroA150',
        'pr76',
        'u159',
        'ulysses16',
        'ulysses22'
    ],
    'optimum_value':[
        2579,
        10628,
        7542,
        118282,
        3323,
        6110,
        6528,
        426,
        629,
        538,
        20749,
        26524,
        108159,
        42080,
        6859,
        7013
    ]
}
asym_data={
    'filename':[
        'br17',
        'ft53',
        'ft70',
        'ftv33',
        'ftv35',
        'ftv38',
        'ftv44',
        'ftv47',
        'p43',
        'ry48p'
    ],
    'optimum_value':[
        39,
        6905,
        38673,
        1286,
        1473,
        1530,
        1613,
        1776,
        5620,
        14422
    ]
}

sym_optimum = pd.DataFrame(sym_data)
sym_optimum.to_csv("sym_baseline_results.csv")

asym_optimum = pd.DataFrame(asym_data)
asym_optimum.to_csv("asym_baseline_results.csv")

# Getting plots

In [11]:
symDIRS = [
    'results_random_start_no_checks/',
    'sym_results_CNA_start_no_checks/',
    'sym_results_CNA_start_ckecks_keep_best/',
    'sym_results_CNA_start_checks_no_keep_best/'
]
asymDIRS = [
    'results_random_start_no_checks/',
    'asym_results_CNA_start_no_checks/',
    'asym_results_CNA_start_checks_keep_best/',
    'asym_results_CNA_start_checks_no_keep_best/'
]

## Symmetric results

In [71]:
DIR = symDIRS[3] #select the directory form the list before

EXT = ".tsp"
sym_res = pd.read_csv(DIR+"Symmetrical_results.csv",index_col=0)

files   = sym_res['filename'].unique()
n_iter  = sym_res['iterations'].unique()
n_pop   = sym_res['pop_size'].unique()

for file in files:
    file_opt = float(sym_optimum[sym_optimum['filename'] == file[:-4]]['optimum_value'])
    plt.figure(figsize=(12,8))
    for pop in n_pop:

        arr = np.array(sym_res.loc[
            (sym_res['filename'] == file)
            &
            (sym_res['pop_size'] == pop)
            ]['global_best'])
        arr = arr / file_opt

        plt.plot(n_iter,arr,'o-', label=f"{pop} individuals, best {np.min(arr):.2f} x optimum path")
        
    plt.xticks(n_iter)
    plt.legend()
    # plt.show()
    plt.savefig("plots/iterations_random_start_" + file[:-4] + ".jpg")
    plt.close()

    plt.figure(figsize=(12,8))
    for iter in n_iter:

        arr = np.array(sym_res.loc[
            (sym_res['filename'] == file)
            &
            (sym_res['iterations'] == iter)
            ]['global_best'])
        arr = arr / file_opt

        plt.plot(n_pop,arr,'o-', label=f"{iter} generations, best {np.min(arr):.2f} x optimum path")
        
    plt.xticks(n_pop)
    plt.legend()
    # plt.show()
    plt.savefig("plots/population_random_start_" + file[:-4] + ".jpg")
    plt.close()


## Asymmetric results

In [72]:
DIR = asymDIRS[3] #select the directory form the list before

EXT = ".atsp"
asym_res =  pd.read_csv(DIR+"Asymmetrical_results.csv",index_col=0)

files   = asym_res['filename'].unique()
n_iter  = asym_res['iterations'].unique()
n_pop   = asym_res['pop_size'].unique()

for file in files:
    file_opt = float(asym_optimum[asym_optimum['filename'] == file[:-5]]['optimum_value'])
    plt.figure(figsize=(12,8))
    for pop in n_pop:

        arr = np.array(asym_res.loc[
            (asym_res['filename'] == file)
            &
            (asym_res['pop_size'] == pop)
            ]['global_best'])
        arr = arr / file_opt

        plt.plot(n_iter,arr,'o-', label=f"{pop} individuals, best {np.min(arr):.2f} x optimum path")
        
    plt.xticks(n_iter)
    plt.legend()
    # plt.show()
    plt.savefig("plots/random_start_" + file[:-5] + "_iterations.jpg")
    plt.close()

    plt.figure(figsize=(12,8))
    for iter in n_iter:

        arr = np.array(asym_res.loc[
            (asym_res['filename'] == file)
            &
            (asym_res['iterations'] == iter)
            ]['global_best'])
        arr = arr / file_opt

        plt.plot(n_pop,arr,'o-', label=f"{iter} generations, best {np.min(arr):.2f} x optimum path")
        
    plt.xticks(n_pop)
    plt.legend()
    # plt.show()
    plt.savefig("plots/random_start_" + file[:-5] + "_population.jpg")
    plt.close()


# Getting tables 

In [67]:
symDIRS = [
    'results_random_start_no_checks/',
    'sym_results_CNA_start_no_checks/',
    'sym_results_CNA_start_checks_keep_best/',
    'sym_results_CNA_start_checks_no_keep_best/'
]
asymDIRS = [
    'results_random_start_no_checks/',
    'asym_results_CNA_start_no_checks/',
    'asym_results_CNA_start_checks_keep_best/',
    'asym_results_CNA_start_checks_no_keep_best/'
]



In [68]:
sym         = pd.DataFrame(columns=['filename', 'iterations', 'pop_size', 'global_best'])

for dir in symDIRS:

    data = []
    sym_res =  pd.read_csv(dir+"Symmetrical_results.csv",index_col=0)

    for name in sym_res['filename'].unique():   
        index = np.argmin(sym_res[sym_res['filename']==name]['global_best'])
        row = sym_res[sym_res['filename']==name].iloc[[index]]
        row['algorithm'] = dir
        sym = pd.concat([sym, row], ignore_index=True)

final=pd.DataFrame()
for name in sym_res['filename'].unique():  
    temp = sym[sym['filename']==name]
    temp = temp.iloc[[np.argmin(temp['global_best'])]]

    final = pd.concat([final, temp],ignore_index=True)

final.to_csv("best_symmetric.csv")
final

,filename,iterations,pop_size,global_best,algorithm
0,u159.tsp,10000,10,49865.762716,sym_results_CNA_start_checks_keep_best/
1,eil51.tsp,10000,40,451.322171,sym_results_CNA_start_no_checks/
2,ulysses16.tsp,1000,40,6809.0,results_random_start_no_checks/
3,kroA150.tsp,10000,40,32007.299471,sym_results_CNA_start_no_checks/
4,ulysses22.tsp,10000,40,6981.0,results_random_start_no_checks/
5,ch130.tsp,10000,40,7137.555994,sym_results_CNA_start_checks_no_keep_best/
6,eil101.tsp,10000,40,735.740404,sym_results_CNA_start_checks_keep_best/
7,pr76.tsp,10000,40,126217.480166,sym_results_CNA_start_no_checks/
8,berlin52.tsp,10000,40,8272.456083,sym_results_CNA_start_checks_keep_best/
9,burma14.tsp,10000,10,3454.0,results_random_start_no_checks/


In [69]:
asym        = pd.DataFrame(columns=['filename', 'iterations', 'pop_size', 'global_best'])

for dir in asymDIRS:

    data = []
    asym_res =  pd.read_csv(dir+"Asymmetrical_results.csv",index_col=0)

    for name in asym_res['filename'].unique():   
        index = np.argmin(asym_res[asym_res['filename']==name]['global_best'])
        row = asym_res[asym_res['filename']==name].iloc[[index]]
        row['algorithm'] = dir
        asym = pd.concat([asym, row], ignore_index=True)

final=pd.DataFrame()
for name in asym_res['filename'].unique():  
    temp = asym[asym['filename']==name]
    temp = temp.iloc[[np.argmin(temp['global_best'])]]

    final = pd.concat([final, temp],ignore_index=True)

final.to_csv("best_asymmetric.csv")
final

,filename,iterations,pop_size,global_best,algorithm
0,ftv33.atsp,10000,40,1375,asym_results_CNA_start_checks_no_keep_best/
1,ftv38.atsp,10000,40,1650,asym_results_CNA_start_no_checks/
2,p43.atsp,10000,40,5623,asym_results_CNA_start_checks_keep_best/
3,br17.atsp,1000,40,39,results_random_start_no_checks/
4,ftv47.atsp,10000,40,1997,results_random_start_no_checks/
5,ry48p.atsp,10000,40,15231,asym_results_CNA_start_no_checks/
6,ft70.atsp,10000,40,42456,asym_results_CNA_start_checks_keep_best/
7,ftv35.atsp,10000,40,1503,asym_results_CNA_start_checks_keep_best/
8,ftv44.atsp,10000,40,1752,asym_results_CNA_start_checks_keep_best/
9,ft53.atsp,10000,40,7631,results_random_start_no_checks/


In [70]:
rows = np.array(sym['filename'].unique())
for dir in symDIRS:
    i=0
    data = []
    sym_res =  pd.read_csv(dir+"Symmetrical_results.csv",index_col=0)

    for name in sym_res['filename'].unique():   
        rows[i] = sym_res[sym_res['filename']==name]
        rows[i]['algorithm'] = dir
        rows[i]['optimum'] = np.min(sym_optimum[sym_optimum['filename']==name[:-4]]['optimum_value'])

        rows[i][['iterations','pop_size','algorithm','global_best','optimum']].to_csv(f"{name[:-4]}.csv")

        

C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\538535495.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[i]['algorithm'] = dir
C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\538535495.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[i]['optimum'] = np.min(sym_optimum[sym_optimum['filename']==name[:-4]]['optimum_value'])
C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\538535495.py:9: SettingWithCopyWarning: 
A value is trying to be set on a 

In [71]:
rows = np.array(sym['filename'].unique())
print(len(sym['filename'].unique()))
for dir in asymDIRS:
    i=0
    data = []
    asym_res =  pd.read_csv(dir+"Asymmetrical_results.csv",index_col=0)

    for name in asym_res['filename'].unique():   
        rows[i] = asym_res[asym_res['filename']==name]
        rows[i]['algorithm'] = dir
        rows[i]['optimum']=np.min(asym_optimum[asym_optimum['filename']==name[:-5]]['optimum_value'])
        print(rows)

        rows[i][['iterations','pop_size','algorithm','global_best','optimum']].to_csv(f"{name[:-5]}.csv")


C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[i]['algorithm'] = dir


16
[      filename  iterations  pop_size  global_best  \
 0   ftv33.atsp         100         5         2856
 1   ftv33.atsp         100        10         2755
 2   ftv33.atsp         100        20         2650
 3   ftv33.atsp         100        40         2754
 4   ftv33.atsp        1000         5         2045
 5   ftv33.atsp        1000        10         2480
 6   ftv33.atsp        1000        20         1797
 7   ftv33.atsp        1000        40         1708
 8   ftv33.atsp       10000         5         1885
 9   ftv33.atsp       10000        10         1694
 10  ftv33.atsp       10000        20         1540
 11  ftv33.atsp       10000        40         1448

                           algorithm  optimum
 0   results_random_start_no_checks/     1286
 1   results_random_start_no_checks/     1286
 2   results_random_start_no_checks/     1286
 3   results_random_start_no_checks/     1286
 4   results_random_start_no_checks/     1286
 5   results_random_start_no_checks/     1286
 6   res

C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[i]['optimum']=np.min(asym_optimum[asym_optimum['filename']==name[:-5]]['optimum_value'])
C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[i]['algorithm'] = dir
C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:11: SettingWithCopyWarning: 
A value is trying to be set 

[      filename  iterations  pop_size  global_best  \
 60  ry48p.atsp         100         5        39444
 61  ry48p.atsp         100        10        36736
 62  ry48p.atsp         100        20        32552
 63  ry48p.atsp         100        40        27419
 64  ry48p.atsp        1000         5        24411
 65  ry48p.atsp        1000        10        20517
 66  ry48p.atsp        1000        20        20790
 67  ry48p.atsp        1000        40        19034
 68  ry48p.atsp       10000         5        21776
 69  ry48p.atsp       10000        10        20709
 70  ry48p.atsp       10000        20        22636
 71  ry48p.atsp       10000        40        16517

                           algorithm  optimum
 60  results_random_start_no_checks/    14422
 61  results_random_start_no_checks/    14422
 62  results_random_start_no_checks/    14422
 63  results_random_start_no_checks/    14422
 64  results_random_start_no_checks/    14422
 65  results_random_start_no_checks/    14422
 66  result

C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[i]['algorithm'] = dir
C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[i]['optimum']=np.min(asym_optimum[asym_optimum['filename']==name[:-5]]['optimum_value'])
C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:10: SettingWithCopyWarning: 
A value is trying to be set 

...

C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[i]['algorithm'] = dir
C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[i]['optimum']=np.min(asym_optimum[asym_optimum['filename']==name[:-5]]['optimum_value'])
C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:10: SettingWithCopyWarning: 
A value is trying to be set 

[      filename  iterations  pop_size  global_best  \
 12  ftv38.atsp         100         5         2627
 13  ftv38.atsp         100        10         2674
 14  ftv38.atsp         100        20         2674
 15  ftv38.atsp         100        40         2514
 16  ftv38.atsp        1000         5         2654
 17  ftv38.atsp        1000        10         2279
 18  ftv38.atsp        1000        20         2332
 19  ftv38.atsp        1000        40         2046
 20  ftv38.atsp       10000         5         2245
 21  ftv38.atsp       10000        10         1962
 22  ftv38.atsp       10000        20         1820
 23  ftv38.atsp       10000        40         1664

                                       algorithm  optimum
 12  asym_results_CNA_start_checks_no_keep_best/     1530
 13  asym_results_CNA_start_checks_no_keep_best/     1530
 14  asym_results_CNA_start_checks_no_keep_best/     1530
 15  asym_results_CNA_start_checks_no_keep_best/     1530
 16  asym_results_CNA_start_checks_no_keep_

C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[i]['algorithm'] = dir
C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[i]['optimum']=np.min(asym_optimum[asym_optimum['filename']==name[:-5]]['optimum_value'])
C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:10: SettingWithCopyWarning: 
A value is trying to be set 

[       filename  iterations  pop_size  global_best  \
 96   ftv44.atsp         100         5         2741
 97   ftv44.atsp         100        10         2741
 98   ftv44.atsp         100        20         2741
 99   ftv44.atsp         100        40         2693
 100  ftv44.atsp        1000         5         2741
 101  ftv44.atsp        1000        10         2688
 102  ftv44.atsp        1000        20         2422
 103  ftv44.atsp        1000        40         2607
 104  ftv44.atsp       10000         5         2641
 105  ftv44.atsp       10000        10         2433
 106  ftv44.atsp       10000        20         2243
 107  ftv44.atsp       10000        40         1781

                                        algorithm  optimum
 96   asym_results_CNA_start_checks_no_keep_best/     1613
 97   asym_results_CNA_start_checks_no_keep_best/     1613
 98   asym_results_CNA_start_checks_no_keep_best/     1613
 99   asym_results_CNA_start_checks_no_keep_best/     1613
 100  asym_results_CNA_st

C:\Users\Tomás Catalán López\AppData\Local\Temp\ipykernel_17268\1183667118.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows[i]['optimum']=np.min(asym_optimum[asym_optimum['filename']==name[:-5]]['optimum_value'])


In [72]:
np.min(sym_optimum[sym_optimum['filename']=='bier127']['optimum_value'])

118282

In [7]:

a = range(3)
a

1 3
2 4
3 5


range(0, 3)